DATA PREPARATION - EMBEDDINGS

In [2]:
##### Run this cell one time only
from train_test_split import train_test_set_split

FILE_PATH = r'pre_train_splicing_prediction.csv'
CHROM_COLUMN = 'CHROM'
TEST_CHROMS = ['chr20', 'chr21']
SPLICE_COLUMN = 'Splicing_types'

train, test = train_test_set_split(FILE_PATH, CHROM_COLUMN, TEST_CHROMS, SPLICE_COLUMN)

File: pre_train_splicing_prediction.csv (25325376 rows)

--- DONE ---
Train_val set size: 24387079 rows.
Test set size: 938297 rows.

--- 'CHROM' ---

Train_val set distribution:
CHROM
chr1     0.1038
chr2     0.0805
chr3     0.0677
chr11    0.0606
chr19    0.0602
chr17    0.0597
chr12    0.0541
chr7     0.0501
chr10    0.0470
chr6     0.0453
chr16    0.0452
chr9     0.0442
chr5     0.0423
chr4     0.0410
chr8     0.0350
chrX     0.0339
chr15    0.0332
chr14    0.0331
chr22    0.0266
chr13    0.0185
chr18    0.0157
chrY     0.0022
Name: proportion, dtype: float64

Test set distribution:
CHROM
chr20    0.721
chr21    0.279
Name: proportion, dtype: float64

--- 'Splicing_types' ---
Original distribution:
Splicing_types
0    0.9804
1    0.0099
2    0.0097
Name: proportion, dtype: float64

Train_val set distribution:
Splicing_types
0    0.9804
1    0.0099
2    0.0097
Name: proportion, dtype: float64

Test set distribution:
Splicing_types
0    0.9814
1    0.0094
2    0.0092
Name: proportion

### Downscale class 0 from test dataset

In [1]:
import pandas as pd

df = pd.read_csv(r'D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\test_data.csv')
df.value_counts('Splicing_types')

Splicing_types
0    920809
1      8822
2      8666
Name: count, dtype: int64

In [2]:
def undersample_by_ratio(df, label_col='Splicing_types', ratio_0=10, random_state=42):
    """
    Hàm cắt giảm Class 0 theo tỉ lệ mong muốn so với Class 1.
    Giữ nguyên Class 1 và Class 2.
    """
    # Tách các nhóm dữ liệu
    df_0 = df[df[label_col] == 0]
    df_1 = df[df[label_col] == 1]
    df_2 = df[df[label_col] == 2]
    
    # Lấy số lượng của Class 1 làm mốc (8,822)
    base_count = len(df_1)
    
    # Tính số lượng cần lấy cho Class 0
    target_n_0 = int(base_count * ratio_0)
    
    # Thực hiện lấy mẫu ngẫu nhiên cho Class 0
    df_0_sampled = df_0.sample(n=min(len(df_0), target_n_0), random_state=random_state)
    
    # Gộp lại và xáo trộn (shuffle) thứ tự các dòng
    df_final = pd.concat([df_0_sampled, df_1, df_2]).sample(frac=1, random_state=random_state).reset_index(drop=True)
    
    return df_final

ratios = [10, 4, 2, 1]
datasets = {}

for r in ratios:
    name = f"df_{r}_1_1"
    datasets[name] = undersample_by_ratio(df, ratio_0=r)
    print(f"Tỉ lệ {r}:1:1 -> Tổng số mẫu: {len(datasets[name])}")
    print(datasets[name]['Splicing_types'].value_counts().sort_index())
    print("-" * 30)

Tỉ lệ 10:1:1 -> Tổng số mẫu: 105708
Splicing_types
0    88220
1     8822
2     8666
Name: count, dtype: int64
------------------------------
Tỉ lệ 4:1:1 -> Tổng số mẫu: 52776
Splicing_types
0    35288
1     8822
2     8666
Name: count, dtype: int64
------------------------------
Tỉ lệ 2:1:1 -> Tổng số mẫu: 35132
Splicing_types
0    17644
1     8822
2     8666
Name: count, dtype: int64
------------------------------
Tỉ lệ 1:1:1 -> Tổng số mẫu: 26310
Splicing_types
0    8822
1    8822
2    8666
Name: count, dtype: int64
------------------------------


In [3]:
datasets['df_10_1_1'].to_csv('test_10_1_1.csv', index=False)
datasets['df_4_1_1'].to_csv('test_4_1_1.csv', index=False)
datasets['df_2_1_1'].to_csv('test_2_1_1.csv', index=False)
datasets['df_1_1_1'].to_csv('test_1_1_1.csv', index=False)

### Ratio splitting

In [2]:
##### Run this cell one time only
import pandas as pd
from ratio_split import ratio_splitting

VAL_SIZE = 0.15
CHROM_COLUMN = 'CHROM'
SPLICE_COLUMN = 'Splicing_types'
TRAIN_DATA_PATH = r'raw\train_val_data.csv' # Fill this only if interupted by error (no need to rerun the 1st cell)

try:
    train
except NameError:
    train = None
if train is None:
    train = pd.read_csv(TRAIN_DATA_PATH)

ratio_splitting(train, CHROM_COLUMN, SPLICE_COLUMN, VAL_SIZE)

Available:
 Class 0: 23907991 
Class 1, 242008 
Class 2, 237080
unit = 242008

Generating ratio 9_1_1 ...


c:\Users\Dung\anaconda3\envs\stable\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


  - Creating dataset 1/10
  - Creating dataset 2/10
  - Creating dataset 3/10
  - Creating dataset 4/10
  - Creating dataset 5/10
  - Creating dataset 6/10
  - Creating dataset 7/10
  - Creating dataset 8/10
  - Creating dataset 9/10
  - Creating dataset 10/10

Generating ratio 4_1_1 ...
  - Creating dataset 1/10
  - Creating dataset 2/10
  - Creating dataset 3/10
  - Creating dataset 4/10
  - Creating dataset 5/10
  - Creating dataset 6/10
  - Creating dataset 7/10
  - Creating dataset 8/10
  - Creating dataset 9/10
  - Creating dataset 10/10

Generating ratio 2_1_1 ...
  - Creating dataset 1/10
  - Creating dataset 2/10
  - Creating dataset 3/10
  - Creating dataset 4/10
  - Creating dataset 5/10
  - Creating dataset 6/10
  - Creating dataset 7/10
  - Creating dataset 8/10
  - Creating dataset 9/10
  - Creating dataset 10/10

Generating ratio 1_1_1 ...
  - Creating dataset 1/10
  - Creating dataset 2/10
  - Creating dataset 3/10
  - Creating dataset 4/10
  - Creating dataset 5/10
  -

In [1]:
##### Run this cell one time only
from extract_embed import embed_train_val_folder

DATA_PATH = r"train_val"

embed_train_val_folder(root=DATA_PATH, replace=True) #Automately delete csv files after embedding -> change to False to keep csv files

c:\Users\Dung\anaconda3\envs\stable\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


...Loading Nucleotide Transformer...


Some weights of EsmModel were not initialized from the model checkpoint at InstaDeepAI/nucleotide-transformer-500m-human-ref and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Device = cuda
Root folder = train_val

Processing set: train_val\4_1_1\set_2

Reading CSV: train_val\4_1_1\set_2\train.csv
Extracting embeddings from Nucleotide Transformer... (1230052 sequences)


100%|██████████| 19220/19220 [1:35:01<00:00,  3.37it/s]


...

[saved] train_val\9_1_1\set_8\val_embeddings.pt
[deleted] train_val\9_1_1\set_8\val.csv
Finished embedding set: train_val\9_1_1\set_8

Processing set: train_val\9_1_1\set_9

Reading CSV: train_val\9_1_1\set_9\train.csv
Extracting embeddings from Nucleotide Transformer... (2258586 sequences)


100%|██████████| 35291/35291 [2:53:44<00:00,  3.39it/s]  


[saved] train_val\9_1_1\set_9\train_embeddings.pt
[deleted] train_val\9_1_1\set_9\train.csv

Reading CSV: train_val\9_1_1\set_9\val.csv
Extracting embeddings from Nucleotide Transformer... (398574 sequences)


100%|██████████| 6228/6228 [30:57<00:00,  3.35it/s]


[saved] train_val\9_1_1\set_9\val_embeddings.pt
[deleted] train_val\9_1_1\set_9\val.csv
Finished embedding set: train_val\9_1_1\set_9

[DONE] All embeddings generated.


In [5]:
# Extract test dataset embedding (run only one time)
from extract_embed import process_csv_embedding
from transformers import AutoTokenizer, AutoModel
import torch
NT_MODEL = "InstaDeepAI/nucleotide-transformer-500m-human-ref"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(NT_MODEL)
model = AutoModel.from_pretrained(NT_MODEL).to(DEVICE)
process_csv_embedding(csv_path=r'D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\raw\test_data.csv', tokenizer=tokenizer, model=model, device=DEVICE, replace=False)

Some weights of EsmModel were not initialized from the model checkpoint at InstaDeepAI/nucleotide-transformer-500m-human-ref and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Reading CSV: D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\raw\test_data.csv
Extracting embeddings from Nucleotide Transformer... (938297 sequences)


100%|██████████| 14661/14661 [1:14:47<00:00,  3.27it/s]


[saved] D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\raw\test_data_embeddings.pt


'D:\\Bio_sequence_Research_AITALAB\\train\\task1_splicing_prediction\\data_preparation\\raw\\test_data_embeddings.pt'

In [4]:
# Extract test dataset embedding (run only one time)
from extract_embed import process_csv_embedding
from transformers import AutoTokenizer, AutoModel
import torch
NT_MODEL = "InstaDeepAI/nucleotide-transformer-500m-human-ref"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(NT_MODEL)
model = AutoModel.from_pretrained(NT_MODEL).to(DEVICE)
process_csv_embedding(csv_path=r'D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\test_1_1_1.csv', tokenizer=tokenizer, model=model, device=DEVICE, replace=False)

c:\Users\Dung\anaconda3\envs\stable\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of EsmModel were not initialized from the model checkpoint at InstaDeepAI/nucleotide-transformer-500m-human-ref and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Reading CSV: D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\test_1_1_1.csv
Extracting embeddings from Nucleotide Transformer... (26310 sequences)


100%|██████████| 412/412 [02:02<00:00,  3.36it/s]

[saved] D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\test_1_1_1_embeddings.pt


'D:\\Bio_sequence_Research_AITALAB\\train\\task1_splicing_prediction\\data_preparation\\train_val\\test_1_1_1_embeddings.pt'

In [5]:
# Extract test dataset embedding (run only one time)
from extract_embed import process_csv_embedding
from transformers import AutoTokenizer, AutoModel
import torch
NT_MODEL = "InstaDeepAI/nucleotide-transformer-500m-human-ref"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(NT_MODEL)
model = AutoModel.from_pretrained(NT_MODEL).to(DEVICE)
process_csv_embedding(csv_path=r'D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\test_2_1_1.csv', tokenizer=tokenizer, model=model, device=DEVICE, replace=False)

Some weights of EsmModel were not initialized from the model checkpoint at InstaDeepAI/nucleotide-transformer-500m-human-ref and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Reading CSV: D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\test_2_1_1.csv
Extracting embeddings from Nucleotide Transformer... (35132 sequences)


100%|██████████| 549/549 [02:39<00:00,  3.45it/s]


[saved] D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\test_2_1_1_embeddings.pt


'D:\\Bio_sequence_Research_AITALAB\\train\\task1_splicing_prediction\\data_preparation\\train_val\\test_2_1_1_embeddings.pt'

In [6]:
# Extract test dataset embedding (run only one time)
from extract_embed import process_csv_embedding
from transformers import AutoTokenizer, AutoModel
import torch
NT_MODEL = "InstaDeepAI/nucleotide-transformer-500m-human-ref"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(NT_MODEL)
model = AutoModel.from_pretrained(NT_MODEL).to(DEVICE)
process_csv_embedding(csv_path=r'D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\test_4_1_1.csv', tokenizer=tokenizer, model=model, device=DEVICE, replace=False)

Some weights of EsmModel were not initialized from the model checkpoint at InstaDeepAI/nucleotide-transformer-500m-human-ref and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Reading CSV: D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\test_4_1_1.csv
Extracting embeddings from Nucleotide Transformer... (52776 sequences)


100%|██████████| 825/825 [04:00<00:00,  3.44it/s]


[saved] D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\test_4_1_1_embeddings.pt


'D:\\Bio_sequence_Research_AITALAB\\train\\task1_splicing_prediction\\data_preparation\\train_val\\test_4_1_1_embeddings.pt'

In [7]:
# Extract test dataset embedding (run only one time)
from extract_embed import process_csv_embedding
from transformers import AutoTokenizer, AutoModel
import torch
NT_MODEL = "InstaDeepAI/nucleotide-transformer-500m-human-ref"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(NT_MODEL)
model = AutoModel.from_pretrained(NT_MODEL).to(DEVICE)
process_csv_embedding(csv_path=r'D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\test_10_1_1.csv', tokenizer=tokenizer, model=model, device=DEVICE, replace=False)

Some weights of EsmModel were not initialized from the model checkpoint at InstaDeepAI/nucleotide-transformer-500m-human-ref and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Reading CSV: D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\test_10_1_1.csv
Extracting embeddings from Nucleotide Transformer... (105708 sequences)


100%|██████████| 1652/1652 [07:58<00:00,  3.45it/s]


[saved] D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\test_10_1_1_embeddings.pt


'D:\\Bio_sequence_Research_AITALAB\\train\\task1_splicing_prediction\\data_preparation\\train_val\\test_10_1_1_embeddings.pt'

In [1]:
# Extract test dataset embedding (run only one time)
from extract_embed import process_csv_embedding
from transformers import AutoTokenizer, AutoModel
import torch
NT_MODEL = "InstaDeepAI/nucleotide-transformer-500m-human-ref"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(NT_MODEL)
model = AutoModel.from_pretrained(NT_MODEL).to(DEVICE)
process_csv_embedding(csv_path=r'D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\gtex_test_1_1_1.csv', tokenizer=tokenizer, model=model, device=DEVICE, replace=False)

c:\Users\Dung\anaconda3\envs\stable\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of EsmModel were not initialized from the model checkpoint at InstaDeepAI/nucleotide-transformer-500m-human-ref and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Reading CSV: D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\gtex_test_1_1_1.csv
Extracting embeddings from Nucleotide Transformer... (41283 sequences)


100%|██████████| 646/646 [03:19<00:00,  3.24it/s]


[saved] D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\gtex_test_1_1_1_embeddings.pt


'D:\\Bio_sequence_Research_AITALAB\\train\\task1_splicing_prediction\\data_preparation\\train_val\\gtex_test_1_1_1_embeddings.pt'

In [2]:
# Extract test dataset embedding (run only one time)
from extract_embed import process_csv_embedding
from transformers import AutoTokenizer, AutoModel
import torch
NT_MODEL = "InstaDeepAI/nucleotide-transformer-500m-human-ref"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(NT_MODEL)
model = AutoModel.from_pretrained(NT_MODEL).to(DEVICE)
process_csv_embedding(csv_path=r'D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\gtex_test_2_1_1.csv', tokenizer=tokenizer, model=model, device=DEVICE, replace=False)

Some weights of EsmModel were not initialized from the model checkpoint at InstaDeepAI/nucleotide-transformer-500m-human-ref and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Reading CSV: D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\gtex_test_2_1_1.csv
Extracting embeddings from Nucleotide Transformer... (55059 sequences)


100%|██████████| 861/861 [04:22<00:00,  3.28it/s]


[saved] D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\gtex_test_2_1_1_embeddings.pt


'D:\\Bio_sequence_Research_AITALAB\\train\\task1_splicing_prediction\\data_preparation\\train_val\\gtex_test_2_1_1_embeddings.pt'

In [3]:
# Extract test dataset embedding (run only one time)
from extract_embed import process_csv_embedding
from transformers import AutoTokenizer, AutoModel
import torch
NT_MODEL = "InstaDeepAI/nucleotide-transformer-500m-human-ref"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(NT_MODEL)
model = AutoModel.from_pretrained(NT_MODEL).to(DEVICE)
process_csv_embedding(csv_path=r'D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\gtex_test_4_1_1.csv', tokenizer=tokenizer, model=model, device=DEVICE, replace=False)

Some weights of EsmModel were not initialized from the model checkpoint at InstaDeepAI/nucleotide-transformer-500m-human-ref and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Reading CSV: D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\gtex_test_4_1_1.csv
Extracting embeddings from Nucleotide Transformer... (82611 sequences)


100%|██████████| 1291/1291 [06:33<00:00,  3.28it/s]


[saved] D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\gtex_test_4_1_1_embeddings.pt


'D:\\Bio_sequence_Research_AITALAB\\train\\task1_splicing_prediction\\data_preparation\\train_val\\gtex_test_4_1_1_embeddings.pt'

In [4]:
# Extract test dataset embedding (run only one time)
from extract_embed import process_csv_embedding
from transformers import AutoTokenizer, AutoModel
import torch
NT_MODEL = "InstaDeepAI/nucleotide-transformer-500m-human-ref"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(NT_MODEL)
model = AutoModel.from_pretrained(NT_MODEL).to(DEVICE)
process_csv_embedding(csv_path=r'D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\gtex_test_10_1_1.csv', tokenizer=tokenizer, model=model, device=DEVICE, replace=False)

Some weights of EsmModel were not initialized from the model checkpoint at InstaDeepAI/nucleotide-transformer-500m-human-ref and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Reading CSV: D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\gtex_test_10_1_1.csv
Extracting embeddings from Nucleotide Transformer... (165267 sequences)


100%|██████████| 2583/2583 [13:06<00:00,  3.28it/s]


[saved] D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\gtex_test_10_1_1_embeddings.pt


'D:\\Bio_sequence_Research_AITALAB\\train\\task1_splicing_prediction\\data_preparation\\train_val\\gtex_test_10_1_1_embeddings.pt'

In [1]:
# Extract test dataset embedding (run only one time)
from extract_embed import process_csv_embedding
from transformers import AutoTokenizer, AutoModel
import torch
NT_MODEL = "InstaDeepAI/nucleotide-transformer-500m-human-ref"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(NT_MODEL)
model = AutoModel.from_pretrained(NT_MODEL).to(DEVICE)
process_csv_embedding(csv_path=r'D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\gtex_test_data.csv', tokenizer=tokenizer, model=model, device=DEVICE, replace=False)

c:\Users\Dung\anaconda3\envs\stable\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of EsmModel were not initialized from the model checkpoint at InstaDeepAI/nucleotide-transformer-500m-human-ref and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Reading CSV: D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\gtex_test_data.csv
Extracting embeddings from Nucleotide Transformer... (1402857 sequences)


100%|██████████| 5480/5480 [1:50:28<00:00,  1.21s/it]


[saved] D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\gtex_test_data_embeddings.pt


'D:\\Bio_sequence_Research_AITALAB\\train\\task1_splicing_prediction\\data_preparation\\train_val\\gtex_test_data_embeddings.pt'